In [1]:
#2018 HMDA Edit Testing File Generator
from collections import OrderedDict
from io import StringIO
import json
import os
import pandas as pd
import random

#custom imports
import lar_constraints
import lar_generator
from rules_engine import rules_engine


In [2]:
#2018 Filing Instruction Guide: https://www.consumerfinance.gov/data-research/hmda/static/for-filers/2018/2018-HMDA-FIG.pdf

use_cols = ['name', 'metDivName', 'countyFips', 'geoIdMsa', 'metDivFp', 'smallCounty', 'tracts']
cbsa_cols = ['name', 'metDivName', 'state', 'countyFips', 'county', 'tracts','geoIdMsa', 'metDivFp', 'smallCounty', 
             'stateCode', 'tractDecimal']
cbsas = pd.read_csv('../dependancies/tract_to_cbsa_2015.txt', usecols=use_cols, delimiter='|', 
                    header=None, names=cbsa_cols, dtype=str) #load tract to CBSA data from platform file
cbsas["tractFips"] = cbsas.countyFips + cbsas.tracts
counties = list(cbsas.countyFips)
tracts = list(cbsas.tractFips)

In [3]:
#load schemas for LAR and transmittal sheet
lar_schema_df = pd.DataFrame(json.load(open("../schemas/lar_schema.json", "r")))
ts_schema_df = pd.DataFrame(json.load(open("../schemas/ts_schema.json", "r")))

In [4]:
lar_gen = lar_generator.lar_gen(lar_schema_df, ts_schema_df, counties=counties, tracts=tracts) #instantiate generator
lar_const = lar_constraints.lar_constraints(counties=counties, tracts=tracts)#instantiate constraints

In [5]:
#run constraint functions on row
file_length = 5 #set number of rows in test file
first = True
lei = None

#create list of constraints in lar_constraints object
constraints = [] 
for func in dir(lar_const):
    if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
        constraints.append(func)
        
for i in range(0, file_length):
    print("making new row\n\n\n\n\n\n\n")
    if lei:
        row = lar_gen.make_row(lei=lei)
    else:
        row = lar_gen.make_row() #create new row
    lei = row["lei"]
    diff = [1] #initialize diff for loop
    iters = 1
    while len(diff) > 0:
        row_base = row.copy() #copy row to enable diff
        #apply constraint functions to LAR row
        print("\n\nstarting constraints iteration {iter}\n\n\n\n\n".format(iter=iters))
        for func in constraints:
            print("\n\n")
            row = getattr(lar_const, func)(row)
            #convert initial and copied rows to sets for diff
            initial_row = set(row_base.items())
            changed_row = set(row.items())
            diff = (initial_row - changed_row) #subtract row sets to show changes from constraint funcs
#            print(len(initial_row), len(changed_row)) #check number of fields, should be 110
            #show readout of number of changes made to new LAR row
            print("changes from {func}: {vals}".format(func=str(func), vals=len(diff)))
            #print(row["app_date"], row["action_taken"])
            print(diff)
        iters+=1
    #create first row of dataframe
    if first:
        lar_frame = pd.DataFrame(row, index=[1])
        first = False
        print("finished row\n",lar_frame.columns)
    #add additional rows to dataframe
    else:
        #print("concating")
        new_lar = pd.DataFrame(row, index=[1])
        lar_frame = pd.concat([lar_frame, new_lar], axis=0)
lar_frame.reset_index(inplace=True) #reset index
lar_frame.drop("index", inplace=True, axis=1) #drop additional index column
print(iters)

making new row









starting constraints iteration 0








v610_const
OrderedDict([('record_id', '2'), ('lei', '687WHGDR7MDZRFRTRVGK'), ('uli', '687WHGDR7MDZRFRTRVGKW7XXDLHRHIRGBX1NSOUMQBC70'), ('app_date', '20180422'), ('loan_type', '4'), ('loan_purpose', '1'), ('preapproval', '1'), ('const_method', '2'), ('occ_type', '3'), ('loan_amount', '2413'), ('action_taken', '4'), ('action_date', '20180320'), ('street_address', '1234 Hocus Potato Way'), ('city', 'Tatertown'), ('state', 'UT'), ('zip_code', '84096'), ('county', '11001'), ('tract', '11001000802'), ('app_eth_1', ''), ('app_eth_2', '2'), ('app_eth_3', '11'), ('app_eth_4', '12'), ('app_eth_5', '11'), ('app_eth_code_14', 'P04MMDMCWIXPANAA6IU9FHINW3TAPW2PE5A6B69F785KZBVE'), ('co_app_eth_1', '2'), ('co_app_eth_2', '12'), ('co_app_eth_3', ''), ('co_app_eth_4', '1'), ('co_app_eth_5', '11'), ('co_app_eth_code_14', '5DFSEETGAYGDMBUYE7C77LSPWILC816CT06QEUNCYF16091REQXJ043HDQZUJQ2O8FHAU2BZ'), ('app_eth_basis', '3'), ('co_app_eth_basis',


v651_const
OrderedDict([('record_id', '2'), ('lei', '687WHGDR7MDZRFRTRVGK'), ('uli', '687WHGDR7MDZRFRTRVGKW7XXDLHRHIRGBX1NSOUMQBC70'), ('app_date', '20180422'), ('loan_type', '4'), ('loan_purpose', '1'), ('preapproval', '2'), ('const_method', '2'), ('occ_type', '3'), ('loan_amount', '2413'), ('action_taken', '6'), ('action_date', '20180422'), ('street_address', '1234 Hocus Potato Way'), ('city', 'Tatertown'), ('state', 'UT'), ('zip_code', '84096'), ('county', '11001'), ('tract', '11001000802'), ('app_eth_1', '4'), ('app_eth_2', ''), ('app_eth_3', ''), ('app_eth_4', ''), ('app_eth_5', ''), ('app_eth_code_14', 'P04MMDMCWIXPANAA6IU9FHINW3TAPW2PE5A6B69F785KZBVE'), ('co_app_eth_1', '3'), ('co_app_eth_2', ''), ('co_app_eth_3', ''), ('co_app_eth_4', ''), ('co_app_eth_5', ''), ('co_app_eth_code_14', '5DFSEETGAYGDMBUYE7C77LSPWILC816CT06QEUNCYF16091REQXJ043HDQZUJQ2O8FHAU2BZ'), ('app_eth_basis', '3'), ('co_app_eth_basis', '3'), ('app_race_1', '2'), ('app_race_2', '1'), ('app_race_3', '4'), ('app

changes from v655_const: 1
{('co_app_eth_1', '4')}



v656_const
OrderedDict([('record_id', '2'), ('lei', '687WHGDR7MDZRFRTRVGK'), ('uli', '687WHGDR7MDZRFRTRVGKW7XXDLHRHIRGBX1NSOUMQBC70'), ('app_date', 'NA'), ('loan_type', '4'), ('loan_purpose', '1'), ('preapproval', '2'), ('const_method', '2'), ('occ_type', '3'), ('loan_amount', '2413'), ('action_taken', '6'), ('action_date', '20180422'), ('street_address', '1234 Hocus Potato Way'), ('city', 'Tatertown'), ('state', 'UT'), ('zip_code', '84096'), ('county', '11001'), ('tract', '11001000802'), ('app_eth_1', '4'), ('app_eth_2', ''), ('app_eth_3', ''), ('app_eth_4', ''), ('app_eth_5', ''), ('app_eth_code_14', 'P04MMDMCWIXPANAA6IU9FHINW3TAPW2PE5A6B69F785KZBVE'), ('co_app_eth_1', '3'), ('co_app_eth_2', ''), ('co_app_eth_3', ''), ('co_app_eth_4', ''), ('co_app_eth_5', ''), ('co_app_eth_code_14', '5DFSEETGAYGDMBUYE7C77LSPWILC816CT06QEUNCYF16091REQXJ043HDQZUJQ2O8FHAU2BZ'), ('app_eth_basis', '3'), ('co_app_eth_basis', '3'), ('app_race_1', '2'), 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [6]:
#Quality and Macro field interrelationship constraints:

In [7]:
#Create a sample TS row
#Note: this will need to be more robust to include other federal agencies
# mlo_id needs NA option
#set dummy values for TS row
ts_row_small = OrderedDict()
ts_row_small["record_id"]="1"
ts_row_small["inst_name"]="Ficus Bank"
ts_row_small["calendar_year"]=str(2018)
ts_row_small["calendar_quarter"]="4"
ts_row_small["contact_name"]="Mr. Smug Pockets"
ts_row_small["contact_tel"]="555-555-5555"
ts_row_small["contact_email"]="pockets@ficus.com"
ts_row_small["contact_street_address"]="1234 Ficus Lane"
ts_row_small["office_city"]="Ficusville"
ts_row_small["office_state"]="UT"
ts_row_small["office_zip"]="84096"
ts_row_small["federal_agency"]="9"
ts_row_small["lar_entries"]= str(len(lar_frame))
ts_row_small["tax_id"]="01-0123456"
ts_row_small["lei"]=lar_frame.get_value(0, "lei")

In [8]:
#join LAR and TS rows to make an output file
def write_file(ts_input=None, lar_input=None, directory="../edits_files/", name="passes_all.txt"):
    """Takes a TS row as a dictionary and LAR data as a dataframe. Writes LAR data to file and 
    re-reads it to combine with TS data to make a full file."""
    #make directories for files if they do not exist
    if not os.path.exists(directory):
        os.makedirs(directory)

    #write LAR dataframe to file
    parts_dir = directory+"file_parts/"
    if not os.path.exists(parts_dir):
        os.makedirs(parts_dir)
        
    lar_input.to_csv(parts_dir + "lar_data.txt", sep="|", header=False, index=False, index_label=False)
    #load LAR data as file rows
    with open(parts_dir + "lar_data.txt", 'r') as lar_data:
        lar = lar_data.readlines()

    with open(directory + name, 'w') as final_file:
        final_file.write("|".join(ts_input.values())+"\n")
        for line in lar:
            final_file.write("{line}".format(line=line))

In [9]:
#modify file for testing

lar_fields = []
ts_fields = []
for field in lar_fields:
    lar_frame[field] = "16"

for field in ts_fields:
    ts_row_small[field] = ""

In [10]:
#write sample file to disk
write_file(ts_input=ts_row_small, lar_input=lar_frame) #writes created file to disk
#validator engine uses the default: path="../edits_files/", data_file="passes_all.txt" for data files
validator = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, tracts=tracts, counties=counties) #instantiate edits rules engine

In [11]:
#split TS and LAR using validator function
#validator creates class objects of each of these internally as well
ts_df, lar_df = validator.split_ts_row(path="../edits_files/", data_file="passes_all.txt")


In [12]:
for func in dir(validator):
    if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
        #print("applying:", func)
        getattr(validator, func)()


In [13]:
validator.results

{'s300_1': OrderedDict([('row_type', 'TS'), ('record_id', 'passed')]),
 's300_2': OrderedDict([('row_type', 'LAR'), ('record_id', 'passed')]),
 's301': OrderedDict([('row_type', 'TS'), ('LEI', 'passed')]),
 's302': OrderedDict([('row_type', 'TS'), ('calendar_year', 'passed')]),
 's304': OrderedDict([('row_type', 'TS/LAR'), ('lar_entries', 'passed')]),
 's305': OrderedDict([('row_type', 'LAR'), ('all', 'passed')]),
 'v600': OrderedDict([('row_type', 'LAR'), ('LEI', 'passed')]),
 'v601_1': OrderedDict([('row_type', 'TS'), ('inst_name', 'passed')]),
 'v601_2': OrderedDict([('row_type', 'TS'), ('contact_name', 'passed')]),
 'v601_3': OrderedDict([('row_type', 'TS'), ('contact_email', 'passed')]),
 'v601_4': OrderedDict([('row_type', 'TS'),
              ('contact_street_address', 'passed')]),
 'v601_5': OrderedDict([('row_type', 'TS'), ('office_city', 'passed')]),
 'v602': OrderedDict([('row_type', 'TS'), ('calendar_quarter', 'passed')]),
 'v603': OrderedDict([('row_type', 'TS'), ('contact

In [14]:
validator.lar_df[["co_app_eth_basis","co_app_eth_1", "co_app_eth_2", "co_app_eth_3", "co_app_eth_4", "co_app_eth_5", "uli"]]

,co_app_eth_basis,co_app_eth_1,co_app_eth_2,co_app_eth_3,co_app_eth_4,co_app_eth_5,uli
0,3,3,,,,,687WHGDR7MDZRFRTRVGKW7XXDLHRHIRGBX1NSOUMQBC70
1,3,3,,,,,687WHGDR7MDZRFRTRVGKTE3WUPJYGOQUHE6HWGNHVKJ04
2,3,4,,,,,687WHGDR7MDZRFRTRVGKR9S1CIHRTHPF8OI9IVKV0TB36
3,4,5,,,,,687WHGDR7MDZRFRTRVGKR4Q9J3HNVO53QKIIYRNBXQ850
4,1,2,1,,,,687WHGDR7MDZRFRTRVGK5VEABDQ8XGN8EG8SANOMW0783


In [15]:
validator.lar_df[["app_date", "action_date"]][(validator.lar_df.action_date < validator.lar_df.app_date)]#&((lar_df.action_taken.isin(("1","2","7","8"))))]

,app_date,action_date
0,NA,20180422
1,NA,20181027
2,NA,20180425


In [16]:
validator.s300_1.__doc__

'1) The first row of your file must begin with a 1;.'

In [17]:
from collections import defaultdict


In [18]:
lst = ["s", "r", "t"]
dct = {i:lst[i] for i in range(len(lst))}
dct

{0: 's', 1: 'r', 2: 't'}